In [45]:
!python3 -m pip install google-cloud-bigquery
!pip install google-cloud-bigquery


In [46]:
!pip install google-cloud-bigquery

In [47]:
#import and connect to bigquery
from google.cloud import bigquery

#import operating system package to 
import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/BGE/Google Drive/Big Query/bge-2024-720fd54b5813.json"
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\Users\BGE\Google Drive\Big Query/bge-2024-720fd54b5813.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/BGE/Google Drive/Big Query/bge-2024-a9a17e3beedc.json"

In [55]:
#

def union_new_tables(project_id, dataset_id, materialized_view_id, new_table_prefix):
    # Initialize BigQuery client
    client = bigquery.Client(project = project_id)

    # Get list of existing tables in the dataset
    list_existing_tables = [table.table_id for table in client.list_tables(dataset_id)]

    # Filter out tables that match the new table prefix
    new_tables = [table for table in list_existing_tables if table.startswith(new_table_prefix)]

    if not new_tables:
        print(f"No new tables found with prefix {new_table_prefix}")

    # Construct the SQL query to union all new tables
        #start of creation
    union_query = """
    CREATE OR REPLACE MATERIALIZED VIEW {dataset}.{materialized_view} OPTIONS (max_staleness = INTERVAL 6 HOUR,
    metadata_cache_mode = 'AUTOMATIC') AS """.format(dataset=dataset_id, materialized_view=materialized_view_id)
        #declare existing materialized view in BigQ 
    union_query += " \nSELECT * FROM {dataset}.{base_materialized_view} ".format(dataset=dataset_id, base_materialized_view=materialized_view_id)

        #loop for adding every new tables 
    for table in new_tables:
        union_query += " \nUNION ALL \nSELECT * FROM {dataset}.{new_table}".format(dataset=dataset_id, new_table=table)
    
    # Execute the union query
        #config the query
    job_config = bigquery.QueryJobConfig()
        #use standard SQL
    job_config.use_legacy_sql = False
        #execute and complete the query
    query_job = client.query(union_query, job_config=job_config)
    query_job.result()  # Wait for the job to complete

if __name__ == "__main__": #assuming this script is executing like file in python interpreter (dunder name value is “__main__) or as a module (dunder name value is the name of the file)                    
    project_id = 'bge-2024'
    dataset_id = 'Leads_per_period'
    materialized_view_id = 'materialized_v_leads_union'
    new_table_prefix = 'to_'

union_new_tables(project_id, dataset_id, materialized_view_id, new_table_prefix)


BadRequest: 400 Materialized views can only reference native tables or BigLake tables with cached metadata enabled or Iceberg tables.; reason: invalid, message: Materialized views can only reference native tables or BigLake tables with cached metadata enabled or Iceberg tables.

Location: EU
Job ID: 87a5d59b-63d9-49e8-a3ac-c00d8e28d10c


In [ ]:
from google.cloud import bigquery

def union_new_tables(project_id, dataset_id, materialized_view_id, new_table_prefix):
    # Initialize BigQuery client
    client = bigquery.Client(project=project_id)

    # Get list of existing tables in the dataset
    list_existing_tables = [table.table_id for table in client.list_tables(dataset_id)]

    # Filter out tables that match the new table prefix
    new_tables = [table for table in list_existing_tables if table.startswith(new_table_prefix)]

    if not new_tables:
        print(f"No new tables found with prefix {new_table_prefix}")
        return

    # Construct the SQL query to union all new tables
    union_query = """
    CREATE OR REPLACE MATERIALIZED VIEW `{materialized_view}` AS 
    SELECT * FROM `{materialized_view}` """.format(materialized_view=materialized_view_id)

    # Loop through new tables and add them to the union query
    for table in new_tables:
        union_query += "\nUNION ALL \nSELECT * FROM `{table}`".format(table=table)

    # Execute the union query
    job_config = bigquery.QueryJobConfig()
    job_config.use_legacy_sql = False
    query_job = client.query(union_query, job_config=job_config)
    query_job.result()  # Wait for the job to complete

if __name__ == "__main__":
    project_id = 'bge-2024'
    dataset_id = 'bge-2024.Leads_per_period'
    materialized_view_id = 'bge-2024.Leads_per_period.materialized_v_leads_union'
    new_table_prefix = 'to_'

    union_new_tables(project_id, dataset_id, materialized_view_id, new_table_prefix)


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.